# 잠재 의미 분석(Latent Semantic Analysis, LSA)

BoW에 기반한 DTM이나 TF-IDF는 기본적으로 단어의 빈도 수를 이용한 수치화 방법이기 때문에 단어의 의미를 고려하지 못한다는 단점이 있었습니다. 이를 위한 대안으로 DTM에 잠재된 의미를 이끌어내는 방법으로 잠재 의미 분석(Latent Semantic Analysis, LSA)이라는 방법이 있습니다. 
<br />

이 방법을 이해하기 위해서는 선형대수학의 특이값 분해(Singular Value Decomposition, SVD)를 이해할 필요가 있습니다. 

## 1. 특이값 분해(Singular Value Decomposition, SVD)

시작하기 앞서, 여기서의 특이값 분해는 실수 벡터 공간에 한정하여 내용을 설명함을 명시합니다. SVD란 A가 m x n 행렬일 때, 다음과 같이 3개의 행렬의 곱으로 분해하는 것을 말합니다.
<br />

$$A = U \Sigma V^T$$
- $U$ : $m \times m$ 직교행렬 ($AA^T = U(\Sigma \Sigma^T)U^T$)

- $V$ : $n \times n$ 직교행렬 ($A^TA = V(\Sigma^T \Sigma)V^T$)

- $\Sigma$ : $m \times n$ 직사각 대각행렬
<br />

여기서 직교행렬(orthogonal matrix)이란 자신과 자신의 전치 행렬(transposed matrix)의 곱 또는 이를 반대로 곱한 결과가 단위행렬(identity matrix)이 되는 행렬을 말합니다. 또한 대각행렬(diagonal matrix)이란 주대각선을 제외한 곳의 원소가 모두 0인 행렬을 의미합니다.
<br />

이때 SVD로 나온 대각 행렬의 대각 원소의 값을 행렬 A의 특이값(Singular value)라고 합니다.
<br />

대각 행렬 $\Sigma$의 주대각원소를 행렬 A의 특이값이라고 하며, 이를 $\sigma_1,\; \sigma_2,\; \ldots,\; \sigma_r$이라 표현할 때 이 값은 내림차순으로 정렬되어 있다는 특징을 가집니다.
<br />

## 2. Truncated SVD

![](https://wikidocs.net/images/page/24949/svd%EC%99%80truncatedsvd.PNG)
<br />

Truncated SVD는 대각 행렬 $\Sigma$의 대각 원소의 값 중에서 상위값 t개만 남게 됩니다. 이 방법은 값의 손실이 일어나므로 기존의 행렬 A를 복구할 수 없습니다. 또한, U행렬과 V행렬의 t열까지만 남깁니다. 여기서 t는 우리가 찾고자하는 토픽의 수를 반영한 하이퍼파라미터값입니다. 
<br />

## 3. 잠재 의미 분석(Latent Semantic Analysis, LSA)

기존의 DTM이나 TF-IDF행렬은 단어의 의미를 전혀 고려하지 못한다는 단점을 갖고 있었습니다. LSA는 기본적으로 DTM이나 TF-IDF 행렬에 절단된 SVD를 사용하여 차원을 축소시키고, 단어들의 잠재적인 의미를 끌어낸다는 아이디어를 갖고 있습니다. 
<br />

- 예시 DTM
<br />

| |과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|문서 1|0|0|0|1|0|1|1|0|0|
|문서 2|0|0|0|1|1|0|1|0|0|
|문서 3|0|1|1|0|2|0|0|0|0|
|문서 4|1|0|0|0|0|0|0|1|1|

<br />



In [1]:
# Full SVD
import numpy as np

A = np.array([
    [0,0,0,1,0,1,1,0,0],
    [0,0,0,1,1,0,1,0,0],
    [0,1,1,0,2,0,0,0,0],
    [1,0,0,0,0,0,0,1,1]
])

print("DTM의 크기(shape) : ", np.shape(A))

DTM의 크기(shape) :  (4, 9)


In [2]:
U, s, VT = np.linalg.svd(A, full_matrices=True)
print("행렬 U:")
print(U.round(2))
print("행렬 U의 크기 : ", np.shape(U))

행렬 U:
[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]
행렬 U의 크기 :  (4, 4)


In [6]:
print("Singular Vector :")
print(s.round(2))
print('Singular Vector shape : ', np.shape(s))

Singular Vector :
[2.69 2.05 1.73 0.77]
Singular Vector shape :  (4,)


In [4]:
# Singular vector 값을 대각 행렬에 삽입
S = np.zeros((4,9))

S[:4, :4] = np.diag(s)

print("Diagonal Matrix S : ")
print(S.round(2))

print('S shape: ')
print(np.shape(S))

Diagonal Matrix S : 
[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]
S shape: 
(4, 9)


In [5]:
print('VT :')
print(VT.round(2))

print('VT shape: ')
print(np.shape(VT))

VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]
VT shape: 
(9, 9)


In [8]:
np.dot(np.dot(U, S), VT).round(2)

array([[ 0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.],
       [ 0., -0., -0.,  1.,  1., -0.,  1., -0., -0.],
       [ 0.,  1.,  1.,  0.,  2., -0.,  0.,  0.,  0.],
       [ 1., -0.,  0.,  0., -0.,  0., -0.,  1.,  1.]])

In [9]:
# Truncated SVD
# 특이값 상위 2개만 보존
S = S[:2, :2]

print('Diagonal Matrix S :')
print(S.round(2))

Diagonal Matrix S :
[[2.69 0.  ]
 [0.   2.05]]


In [10]:
U = U[:, :2]
print('U :')
print(U.round(2))

U :
[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


In [11]:
VT = VT[:2, :]
print('VT :')
print(VT.round(2))

VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


In [12]:
A_prime = np.dot(np.dot(U, S), VT)
print(A)
print()
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]

[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.  ]]


## 4. 실습

In [14]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print(len(documents))

11314


In [15]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [16]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [17]:
# preprocessing
news_df = pd.DataFrame({'document' : documents})

news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")

news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x : ' '.join([w for w in x.split() if len(w) > 3]))

news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x : x.lower())
news_df['clean_doc'][1]

C:\Users\Myeong\AppData\Local\Temp\ipykernel_5764\542516165.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [18]:
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x : x.split())
tokenized_doc = tokenized_doc.apply(lambda x : [item for item in x if item not in stop_words])

NameError: name 'tokenized_cot' is not defined

In [19]:
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


In [20]:
# TF-IDF matrix
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

In [21]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000, max_df=0.5, smooth_idf=True)

x = vectorizer.fit_transform(news_df['clean_doc'])

print("TF-IDF shape :", x.shape)

TF-IDF shape : (11314, 1000)


In [22]:
# Topic Modelling

svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(x)
len(svd_model.components_)

20

In [23]:
# svd_model.components_ == VT
np.shape(svd_model.components_)

(20, 1000)

In [24]:
terms = vectorizer.get_feature_names_out()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d :"%(idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n-1:-1]])

get_topics(svd_model.components_, terms)

Topic 1 : [('like', 0.21386), ('know', 0.20046), ('people', 0.19293), ('think', 0.17805), ('good', 0.15128)]
Topic 2 : [('thanks', 0.32888), ('windows', 0.29088), ('card', 0.18069), ('drive', 0.17455), ('mail', 0.15111)]
Topic 3 : [('game', 0.37064), ('team', 0.32443), ('year', 0.28154), ('games', 0.2537), ('season', 0.18419)]
Topic 4 : [('drive', 0.53324), ('scsi', 0.20165), ('hard', 0.15628), ('disk', 0.15578), ('card', 0.13994)]
Topic 5 : [('windows', 0.40399), ('file', 0.25436), ('window', 0.18044), ('files', 0.16078), ('program', 0.13894)]
Topic 6 : [('chip', 0.16114), ('government', 0.16009), ('mail', 0.15625), ('space', 0.1507), ('information', 0.13562)]
Topic 7 : [('like', 0.67086), ('bike', 0.14236), ('chip', 0.11169), ('know', 0.11139), ('sounds', 0.10371)]
Topic 8 : [('card', 0.46633), ('video', 0.22137), ('sale', 0.21266), ('monitor', 0.15463), ('offer', 0.14643)]
Topic 9 : [('know', 0.46047), ('card', 0.33605), ('chip', 0.17558), ('government', 0.1522), ('video', 0.14356)]

## 5. Pros and Cons of LSA(LSA의 장단점)

LSA는 쉽고 빠르게 구현이 가능할 뿐 아니라 단어의 잠재적인 의미를 이끌어낼 수 있어 문서의 유사도 계산 등에서 좋은 성능을 보여준다는 장점을 갖고 있습니다. 하지만 SVD의 특성상 이미 계산된 LSA에 새로운 데이터를 추가하여 계산하려고 하면 보통 처음부터 다시 계산해야 합니다. 즉, 새로운 정보에 대한 업데이트가 어렵습니다.

# Latent Dirichlet Allocation, LDA

아래와 같은 예시 문서가 있다고 가정하고 LDA에 대해 알아보겠습니다.
<br />

- 문서 1 : 저는 사과랑 바나나를 먹어요

- 문서 2 : 우리는 귀여운 강아지가 좋아요

- 문서 3 : 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요
<br />

LDA를 수행할 때 문서 집합에서 토픽이 몇 개가 존재할지 가정하는 것은 사용자가 해야 할 일입니다. 여기서는 LDA에 2개의 토픽을 찾으라고 요청하겠습니다. 토픽의 개수를 의미하는 변수를 k라고 하였을 때, k를 2로 한다는 의미입니다. 
<br />

LDA는 각 문서의 토픽 분포와 각 토픽 내의 단어 분포를 추정합니다.
<br />

<각 문서의 토픽 분포>  
- 문서1 : 토픽 A 100%

- 문서2 : 토픽 B 100%

- 문서3 : 토픽 B 60%, 토픽 A 40%
<br />

<각 토픽의 단어 분포>  
- 토픽 A : 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜찍하고 0%, 좋아요 0%

- 토픽 B : 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%
<br />

LDA는 토픽의 제목을 정해주지 않지만, 이 시점에서 알고리즘의 사용자는 위 결과로부터 두 토픽이 각각 과일에 대한 토픽과 강아지에 대한 토픽이라고 판단해볼 수 있습니다. 

## 2. LDA의 가정

LDA는 문서의 집합으로부터 어떤 토픽이 존재하는지를 알아내기 위한 알고리즘입니다. LDA는 앞서 배운 빈도수 기반의 표현 방법인 BoW의 행렬 DTM, TF-IDF 행렬을 입력으로 하는데, 이로부터 알 수 있는 사실은 LDA는 단어의 순서는 신경쓰지 않는다는 것입니다.
<br />

LDA는 문서들로부터 토픽을 뽑아내기 위해서 이러한 가정을 염두에두고 있습니다. 모든 문서 하나, 하나가 작성될 때 그 문서의 작성하는 이러한 생각을 했습니다. '나는 이 문서를 작성하기 위해서 이런 주제들을 넣을거고, 이런 주제들을 위해서는 이런 단어들을 넣을 거야'. 조금 더 구체적으로 알아보겠습니다. 각각의 문서는 다음과 같은 과정을 거쳐서 작성되었다고 가정합니다.
<br />

1) 문서에 사용할 단어의 개수 N을 정합니다.

2) 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정합니다.

3) 문서에 사용할 각 단어를 정합니다.

    3-1) 토픽 분포에서 토픽 T를 확률적으로 고릅니다.

    3-2) 선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고릅니다.
<br />

이러한 과정을 통해 문서가 작성되었다는 가정 하에 LDA는 토픽을 뽑아내기 위하여 위 과정을 역으로 추적하는 역공학(reverse engineering)을 수행합니다.

## 3. LDA 수행

LDA의 수행 과정은 아래와 같습니다.
<br />

1) 사용자는 알고리즘에게 토픽의 개수 k를 알려줍니다.  

    - 앞서 말하였듯이 LDA에게 토픽의 개수를 알려주는 역할은 사용자의 역할입니다. LDA는 토픽의 개수 k를 입력바드면, k개의 토픽이 M개의 전체 문서에 걸쳐 분포되어 있다고 가정합니다.

<br />

2) 모든 단어를 k개 중 하나의 토픽에 할당합니다.  
<br />

    - 이제 LDA는 모든 문서의 모든 단어에 대해서 k개 중 하나의 토픽을 랜덤으로 할당합니다. 이 작업이 끝나면 각 문서는 토픽을 가지며, 토픽은 단어 분포를 가지는 상태입니다. 물론 랜덤으로 할당하였기 때문에 사실 이 결과는 전부 틀린 상태입니다. 만약 한 단어가 한 문서에서 2회 이상 등장하였다면, 각 단어는 서로 다른 토픽에 할당되었을 수도 있습니다.
    
<br />

3) 이제 모든 문서의 모든 단어에 대해서 아래의 사항을 반복합니다.(iterative)
<br />

    3-1) 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정합니다. 이에 따라 단어 w는 아래의 두 가지 기준에 따라서 토픽이 재할당됩니다. 
<br /> 

    - $p(\text{topic t}| \text{document d})$ : 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율  

    - $p(\text{word w}| \text{topic t})$ : 각 토픽들 t에서 해당 단어 w의 분포  

<br />

이를 반복하면, 모든 할당이 완료된 수렴 상태가 됩니다. 두 가지 기준이 어떤 의미인지 예를 들어보겠습니다. 설명의 편의를 위해서 두 개의 문서라는 새로운 예를 사용합니다.

![](https://wikidocs.net/images/page/30708/lda1.PNG)
<br />

위 그림은 doc1, doc2를 보여줍니다. 여기서는 doc1의 세 번째 단어 apple의 토픽을 결정하고자 합니다.
<br />

![](https://wikidocs.net/images/page/30708/lda3.PNG)
<br />

우선 첫 번째로 사용하는 기준은 문서 doc1의 단어들이 어떤 토픽에 해당하는지를 봅니다. doc1의 모든 단어들은 50:50의 비율로 할당되어 있기 때문에, apple은 어느 토픽에도 속할 가능성이 있습니다.
<br />

![](https://wikidocs.net/images/page/30708/lda2.PNG)
<br />

두 번째 기준은 단어 apple이 전체 문서에서 어떤 토픽에 할당되어져 있는지를 봅니다. 이 기준에 따르면 단어 apple은 토픽 B에 할당될 가능성이 높습니다. 이러한 두 가지 기준을 참고하여 LDA는 doc1의 apple을 어떤 토픽에 할당할지 결정합니다.

In [25]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [27]:
# word_id : 단어가 정수 인코딩 된 값
# word_frequency : 해당 뉴스에서 해당 단어의 빈도수

from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1])

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [28]:
print(dictionary[66])

faith


In [29]:
len(dictionary)

64281

In [30]:
# LDA 모델 훈련
import gensim
num_topics = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.013*"bike" + 0.008*"engine" + 0.008*"pain" + 0.007*"front"')
(1, '0.013*"nist" + 0.011*"ncsl" + 0.011*"echo" + 0.007*"escape"')
(2, '0.017*"navy" + 0.008*"rows" + 0.006*"reality" + 0.006*"virtual"')
(3, '0.015*"would" + 0.014*"people" + 0.011*"think" + 0.007*"know"')
(4, '0.060*"israel" + 0.037*"israeli" + 0.025*"printf" + 0.025*"arab"')
(5, '0.018*"game" + 0.017*"team" + 0.014*"year" + 0.013*"games"')
(6, '0.019*"period" + 0.010*"play" + 0.010*"power" + 0.009*"pittsburgh"')
(7, '0.021*"space" + 0.008*"nasa" + 0.005*"ground" + 0.005*"data"')
(8, '0.011*"thanks" + 0.011*"would" + 0.011*"drive" + 0.011*"know"')
(9, '0.017*"would" + 0.010*"good" + 0.010*"like" + 0.008*"much"')
(10, '0.012*"said" + 0.012*"people" + 0.011*"like" + 0.011*"know"')
(11, '0.027*"sale" + 0.023*"shipping" + 0.021*"offer" + 0.021*"condition"')
(12, '0.013*"cover" + 0.009*"copies" + 0.008*"rockefeller" + 0.007*"helmet"')
(13, '0.019*"output" + 0.017*"entry" + 0.017*"window" + 0.014*"program"')
(14, '0.040*"a

In [31]:
# 문서별 토픽 보기
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i, '번 째 문서의 topic 비율은', topic_list)

0 번 째 문서의 topic 비율은 [(3, 0.53738713), (4, 0.0498493), (10, 0.15771008), (15, 0.24191818)]
1 번 째 문서의 topic 비율은 [(3, 0.519869), (5, 0.033638295), (6, 0.02737228), (7, 0.031021327), (9, 0.16262648), (10, 0.20702533)]
2 번 째 문서의 topic 비율은 [(3, 0.22499505), (4, 0.12946573), (9, 0.24001081), (10, 0.17392334), (15, 0.2194754)]
3 번 째 문서의 topic 비율은 [(0, 0.055080734), (3, 0.18091357), (5, 0.088360846), (9, 0.23252682), (13, 0.045764364), (19, 0.38621822)]
4 번 째 문서의 topic 비율은 [(5, 0.47409064), (9, 0.49256325)]


In [33]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄식 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x : (x[1]), reverse=True)

        for j, (topic_num, prop_topic) in enumerate(doc):
            if j==0:
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_list]), ignore_index=True)
            else:
                break
    return (topic_table)

In [34]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index()
topictable.columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

C:\Users\Myeong\AppData\Local\Temp\ipykernel_5764\2689692921.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_list]), ignore_index=True)
C:\Users\Myeong\AppData\Local\Temp\ipykernel_5764\2689692921.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_list]), ignore_index=True)
C:\Users\Myeong\AppData\Local\Temp\ipykernel_5764\2689692921.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_list]), ignore_index=True)
C:\Users\Myeong\AppData\Local\Temp\ipykernel_5

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,3,0.5374,"[(3, 0.537395), (4, 0.049849935), (10, 0.15770..."
1,1,3,0.5200,"[(3, 0.52000743), (5, 0.03366713), (6, 0.02737..."
2,2,9,0.2404,"[(3, 0.22400203), (4, 0.12944782), (9, 0.24042..."
3,3,19,0.3862,"[(0, 0.055057142), (3, 0.18085468), (5, 0.0883..."
4,4,9,0.4924,"[(5, 0.47423834), (9, 0.49241552)]"
5,5,17,0.4539,"[(2, 0.047752813), (9, 0.10681931), (10, 0.355..."
6,6,8,0.4599,"[(7, 0.095337406), (8, 0.459853), (9, 0.194931..."
7,7,3,0.4352,"[(3, 0.43522894), (4, 0.06147187), (10, 0.1973..."
8,8,3,0.4685,"[(3, 0.46847826), (13, 0.07165137), (14, 0.119..."
9,9,0,0.4348,"[(0, 0.43483698), (8, 0.15143855), (9, 0.30465..."
